
## Отчет по реализации

https://yandex.ru/dev/market/partner-api/doc/ru/reference/reports/generateGoodsRealizationReport

In [ ]:
import requests as req
import pandas as pd
import matplotlib
import json
import time
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Монтирую образ GOOGLE DRIVE

In [ ]:
time.sleep(60)

In [ ]:
! cp /content/gdrive/MyDrive/1_pet_project/colab_notebooks/tokens_ym.txt /content/tokens_ym
# # Копируем файл с токенами из образа моего GOOGLE DRIVE во временый сервер ноутбука

In [ ]:
time.sleep(20)

In [ ]:
# Читаем файл с токенами
with open('/content/tokens_ym', 'r') as api_key_dct:
    api_key_list = json.load(api_key_dct)

# type(api_key_list)
# Всё хорошо. Даёт список словарей

In [ ]:
api_key_list

In [ ]:
# @title Формирование запроса на получение отчета о реализации и получение ключа доступа к отчету


# Назначаю функцию которая мне в цикле изменит лист на словарь в каждой итеррации
def convert_to_dict(my_list):
    convert_dict = {}
    for i in range(0, len(my_list), 2):
        convert_dict[my_list[i]] = my_list[i + 1]
    return convert_dict

In [ ]:
# Создаю цикл в котором проходим по каждому кабинету и вставляем в реквест campaignId
sheet_with_keys_for_reports = []

for item in api_key_list:
  campaignId = item['campaign_id']
  api_key = item['api_key']
  year = datetime.now().year
  month = datetime.now().month - 1   # за прошлый месяц

  # POST Запрос на получение ключей доступа к отчетам
  body = {"campaignId": campaignId, "year": year, "month" : month}
  url = 'https://api.partner.market.yandex.ru/reports/goods-realization/generate'
  api_key_dict = {'Api-Key': api_key}
  response = req.post(url=url, headers=api_key_dict, json=body)

  if response.json()['status'] == "OK":

    # Если статус ответа ОК, то добавляем к списку апи ключ от кабинета и ключ для доступа к отчету и брэнд:
    iterrable_string = [response.json()['result']['reportId'], [api_key, item['brand']]]

    dict_with_keys = convert_to_dict(iterrable_string)  # Конвертируем в словарь

    # Добавляем словари в список словарей
    sheet_with_keys_for_reports.append(dict_with_keys)
    print(item['client'], item['brand'], "Status", response.json()['status'], sep='  -  ', end='\n')

  else:
    print('----------------------------------------------------------------------------')
    print('ВНИМАНИЕ!!! Данные по кабинету {a} и брэнду {b} : не загрузились и отсутствуют в списке sheet_with_keys_for_reports'.format(a=item['client'], b=item['brand']))

In [ ]:
# response.json()

In [ ]:
# sheet_with_keys_for_reports

In [ ]:
# @title Блок для отладки отдельных кабинетов
# campaignId = --------
# year = datetime.now().year
# month = datetime.now().month - 3   # за прошлый месяц
# body = {"campaignId": campaignId, "year": year, "month" : month}
# url = 'https://api.partner.market.yandex.ru/reports/goods-realization/generate'
# api_Key = {'Api-Key': '-------------'}
# response = req.post(url=url, headers=api_Key, json=body)
# reportId = response.json()['result']['reportId']
# reportId

In [ ]:
# # Для отладки отдельных кабинетов
# url = f'https://api.partner.market.yandex.ru/reports/info/{reportId}'
# headers = {'Api-Key': '-------'} # Создаём словарь для тела запроса

# # Запрашиваем
# realization_report = req.get(url=url, headers=headers)
# realization_report.json()

In [ ]:
# @title Ожидаем пока сформируются отчеты
print("Жду 2 мин")
time.sleep(120)
print("Загружаю")

In [ ]:
# @title Запрос и получение отчета о реализации по ключу.
realization_report_links_list = []
for i in sheet_with_keys_for_reports:
  for key_for_reports, api in i.items():

    # Данные для GET запроса
    # reportId = list(key_for_reports)[0]  # Узнаём метку ключа словаря и передаём её в переменную reportId. Первый вариант кода без метода items()
    reportId = key_for_reports # Узнаём метку ключа словаря и передаём её в переменную reportId через метод .items()
    url = f'https://api.partner.market.yandex.ru/reports/info/{reportId}'

    api_token = api[0] # Из итеррируемого словаря в этом цикле берем значение по ключу словаря, который также является и ключём к отчету
    brand = api[1]
    # print(key_for_reports, api_token, brand)
    headers = {'Api-Key': api_token} # Создаём словарь для тела запроса

    # Запрашиваем
    realization_report = req.get(url=url, headers=headers)

    if 'file' in realization_report.json()['result']: # Если ключ 'file' содержиться в словаре realization_report.json()['result'] то:
      print(brand,':  ', realization_report.json()['result']['file'])
      realization_report_links_list.append(realization_report.json()['result']['file'])
    else:
      print(brand,':  ', 'Нет продаж', realization_report.json()['result'])


In [ ]:
# # Проба работы метода .items()
# for i in sheet_with_keys_for_reports:
#   for k, v in i.items():
#     print(k)
#     print('          ')
#     print(v[0])
#     print('         ')
#     print(v[1])
#     print('----------')
#     print('----------')

In [ ]:
# dct = {'a': 1, 'b': 2, 'c': 3}
# 'a' in dct
# Способ проверки значения в словаре

In [ ]:
# realization_report_links_list

In [ ]:
# pd.read_excel(realization_report_links_list[0], skiprows=13,sheet_name='Товары, переданные в доставку')

In [ ]:
# @title Соединение в один отчет

dataframes = []
for i in range(len(realization_report_links_list)): # Проходим по списку ссылок на отчеты и загружаем каждый отчет. Предобрабатываем и обьединяем в цикле

  # Создаём датафрейм в итерации цикла
  dfn = pd.read_excel(realization_report_links_list[i],skiprows=13,sheet_name='Товары, переданные в доставку')  \
  .drop( # Дропаем ненужные поля
       axis=1,
       columns=['Регистрационный номер таможенной декларации или Регистрационный номер партии товара, подлежащего прослеживаемости (РНПТ)',
           'Наименование организации', 'ИНН', 'КПП', 'Юридический адрес',
           'Статус УПД', 'Дата УПД', 'Номер УПД', 'Дата товарной накладной',
           'Номер товарной накладной', 'Дата счёта-фактуры', 'Номер счёта фактуры']) \
  .dropna(axis=0, thresh=10) # Убираем строку итого

  dataframes.append(dfn) # Добавляем созданный в цикле датафрейм в список датафреймов
  initial_dataframe = pd.concat(dataframes) # Соединяем список датафреймов в один датафрейм

## Получили начальный датафрейм по всем кабинетам

In [ ]:
initial_dataframe.to_excel(f'/content/Товары переданные в доставку_{month}.{year}.xlsx', sheet_name='Товары, переданные в доставку')

In [ ]:
# initial_dataframe.isna().sum()

In [ ]:
# otladka_df = dfn = pd.read_excel(realization_report_links_list[i],skiprows=13,sheet_name='Товары, переданные в доставку')  \
#   .drop( # Дропаем ненужные поля
#        axis=1,
#        columns=['Регистрационный номер таможенной декларации или Регистрационный номер партии товара, подлежащего прослеживаемости (РНПТ)',
#            'Наименование организации', 'ИНН', 'КПП', 'Юридический адрес',
#            'Статус УПД', 'Дата УПД', 'Номер УПД', 'Дата товарной накладной',
#            'Номер товарной накладной', 'Дата счёта-фактуры', 'Номер счёта фактуры']) \
#   .dropna(axis=0, thresh=10) # Убираем строку итого


In [ ]:
# otladka_df.head(15)